In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from user_agents import parse
import numpy as np
from user_agents import parse
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
import lightgbm as lgb
from sklearn.metrics import f1_score

In [37]:
test = pd.read_csv("/home/oleg/.kaggle/competitions/ds-ai-spring-hack/test.csv")
train = pd.read_csv("/home/oleg/.kaggle/competitions/ds-ai-spring-hack/train.csv")

/home/oleg/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# create user agents hash map
ua_dict = {}
for ua in train.user_agent.values:
    if str(ua) != "nan" and ua not in ua_dict:
        ua_dict[ua] = parse(ua)

In [5]:
for ua in test.user_agent.values:
    if str(ua) != "nan" and ua not in ua_dict:
        ua_dict[ua] = parse(ua)

In [38]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1281499 entries, 0 to 1281498
Data columns (total 11 columns):
comment_id           1281499 non-null int64
company_id           1281499 non-null int64
user_id              1281499 non-null int64
product_id           829477 non-null float64
comment              1281499 non-null object
rating               1281499 non-null int64
date_created         1281499 non-null object
user_date_created    1281499 non-null object
user_ip              1281282 non-null object
user_agent           1281280 non-null object
is_fake              1281499 non-null bool
dtypes: bool(1), float64(1), int64(4), object(5)
memory usage: 99.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252099 entries, 0 to 252098
Data columns (total 11 columns):
comment              252099 non-null object
comment_id           252099 non-null int64
company_id           252099 non-null int64
date_created         252099 non-null object
month_created        83026 non-null objec

In [39]:
train.user_agent.fillna("", inplace=True)
train.user_ip.fillna("0.0.0.0", inplace=True)
train["product_id"] = train.product_id.fillna(0).astype(int) 

In [40]:
test.user_agent.fillna("", inplace=True)
test.user_ip.fillna("0.0.0.0", inplace=True)
test["product_id"] = test.product_id.fillna(0).astype(int) 

In [41]:
train["user_date_created_trunc"] = train.user_date_created.apply(lambda x: x.split()[0])
test["user_date_created_trunc"] = test.user_date_created.apply(lambda x: x.split()[0])

In [42]:
train["date_created_trunc"] = train.date_created.apply(lambda x: x.split()[0])
test["date_created_trunc"] = test.date_created.apply(lambda x: x.split()[0])

In [43]:
train['is_mobile'] = train.user_agent.apply(lambda x: \
                                  ua_dict[x].is_mobile if str(x).lower() != "" else False)
train['is_tablet'] = train.user_agent.apply(lambda x: \
                                  ua_dict[x].is_tablet if str(x).lower() != "" else False)
train['is_pc'] = train.user_agent.apply(lambda x: \
                                  ua_dict[x].is_pc if str(x).lower() != "" else False)

In [44]:
train['browser_family'] = train.user_agent.apply(lambda x: \
                                  ua_dict[x].browser.family if str(x).lower() != "" else "")
train['browser_version_string'] = train.user_agent.apply(lambda x: \
                                  ua_dict[x].browser.version_string if str(x).lower() != "" else "")
train['os_family'] = train.user_agent.apply(lambda x: \
                                  ua_dict[x].os.family if str(x).lower() != "" else "")
train['os_version_string'] = train.user_agent.apply(lambda x: \
                                  ua_dict[x].os.version_string if str(x).lower() != "" else "")
train['device_brand'] = train.user_agent.apply(lambda x: \
                                  ua_dict[x].device.brand if str(x).lower() != "" else "")
train['device_model'] = train.user_agent.apply(lambda x: \
                                  ua_dict[x].device.model if str(x).lower() != "" else "")

In [219]:
ua_columns =['is_mobile',
 'is_tablet',
 'is_pc',
 'browser_family',
 'browser_version_string',
 'os_family',
 'os_version_string',
 'device_brand',
 'device_model']

In [45]:
train["user_ip_trunc"] = train.user_ip.apply(lambda x: ".".join(x.split('.')[:2]))
test["user_ip_trunc"] = test.user_ip.apply(lambda x: ".".join(x.split('.')[:2]))

In [46]:
def add_mean_count_on(df1, df2, prop):
    mean = df1.groupby(prop).is_fake.mean()
    mean.name = prop + '_mean_fake'
    count = df1.groupby(prop).is_fake.sum()
    count.name = prop + '_count_fake'
    
    newdf = df2.join(mean, on=prop).join(count, on=prop)
    newdf[prop + '_mean_fake'].fillna(0., inplace=True)
    newdf[prop + '_count_fake'].fillna(0., inplace=True)
    
    return newdf

In [47]:
for prop in ['user_id', 'company_id', 'user_ip_trunc', 'user_date_created_trunc', 'date_created_trunc', 'product_id']:
    train = add_mean_count_on(train, train, prop)
    test = add_mean_count_on(train, test, prop)

In [309]:
# for prop in ['browser_version_string']:
#     # train = add_mean_count_on(train, train, prop)
#     test = add_mean_count_on(train, test, prop)

In [50]:
train.columns

Index(['comment_id', 'company_id', 'user_id', 'product_id', 'comment',
       'rating', 'date_created', 'user_date_created', 'user_ip', 'user_agent',
       'is_fake', 'user_date_created_trunc', 'date_created_trunc', 'is_mobile',
       'is_tablet', 'is_pc', 'browser_family', 'browser_version_string',
       'os_family', 'os_version_string', 'device_brand', 'device_model',
       'user_ip_trunc', 'user_id_mean_fake', 'user_id_count_fake',
       'company_id_mean_fake', 'company_id_count_fake',
       'user_ip_trunc_mean_fake', 'user_ip_trunc_count_fake',
       'user_date_created_trunc_mean_fake',
       'user_date_created_trunc_count_fake', 'date_created_trunc_mean_fake',
       'date_created_trunc_count_fake', 'product_id_mean_fake',
       'product_id_count_fake'],
      dtype='object')

In [295]:
train[['is_fake','is_mobile_mean_fake', 'is_mobile_count_fake',
       'is_tablet_mean_fake', 'is_tablet_count_fake', 'is_pc_mean_fake',
       'is_pc_count_fake', 'browser_family_mean_fake',
       'browser_family_count_fake', 'browser_version_string_mean_fake',
       'browser_version_string_count_fake', 'os_family_mean_fake',
       'os_family_count_fake', 'os_version_string_mean_fake',
       'os_version_string_count_fake', 'device_brand_mean_fake',
       'device_brand_count_fake', 'device_model_mean_fake',
       'device_model_count_fake']].corr()

,is_fake,is_mobile_mean_fake,is_mobile_count_fake,is_tablet_mean_fake,is_tablet_count_fake,is_pc_mean_fake,is_pc_count_fake,browser_family_mean_fake,browser_family_count_fake,browser_version_string_mean_fake,browser_version_string_count_fake,os_family_mean_fake,os_family_count_fake,os_version_string_mean_fake,os_version_string_count_fake,device_brand_mean_fake,device_brand_count_fake,device_model_mean_fake,device_model_count_fake
is_fake,1.000000,0.019845,0.019845,0.010494,0.010494,0.023010,0.023010,0.075239,0.039869,0.200146,0.031574,0.095111,-0.010582,0.026747,0.023192,-0.003497,-0.019768,0.010561,-0.010757
is_mobile_mean_fake,0.019845,1.000000,1.000000,-0.052148,-0.052148,0.846956,0.846956,0.260055,0.426011,-0.066708,-0.059195,0.205046,0.363048,0.719978,0.806625,-0.195847,-0.701491,-0.021942,-0.398223
is_mobile_count_fake,0.019845,1.000000,1.000000,-0.052148,-0.052148,0.846956,0.846956,0.260055,0.426011,-0.066708,-0.059195,0.205046,0.363048,0.719978,0.806625,-0.195847,-0.701491,-0.021942,-0.398223
is_tablet_mean_fake,0.010494,-0.052148,-0.052148,1.000000,1.000000,0.475221,0.475221,-0.006011,0.008175,0.011999,0.006397,0.116211,0.212059,0.406816,0.453709,-0.162911,-0.445101,-0.072112,-0.206067
is_tablet_count_fake,0.010494,-0.052148,-0.052148,1.000000,1.000000,0.475221,0.475221,-0.006011,0.008175,0.011999,0.006397,0.116211,0.212059,0.406816,0.453709,-0.162911,-0.445101,-0.072112,-0.206067
is_pc_mean_fake,0.023010,0.846956,0.846956,0.475221,0.475221,1.000000,1.000000,0.224091,0.376426,-0.052739,-0.049034,0.241932,0.431894,0.849761,0.950956,-0.259744,-0.855013,-0.057887,-0.458791
is_pc_count_fake,0.023010,0.846956,0.846956,0.475221,0.475221,1.000000,1.000000,0.224091,0.376426,-0.052739,-0.049034,0.241932,0.431894,0.849761,0.950956,-0.259744,-0.855013,-0.057887,-0.458791
browser_family_mean_fake,0.075239,0.260055,0.260055,-0.006011,-0.006011,0.224091,0.224091,1.000000,0.529897,0.296407,0.331310,0.092259,0.157677,0.188561,0.209961,-0.040466,-0.196775,-0.001462,-0.139624
browser_family_count_fake,0.039869,0.426011,0.426011,0.008175,0.008175,0.376426,0.376426,0.529897,1.000000,0.083551,0.494441,0.147622,0.161075,0.324795,0.369040,-0.066563,-0.334852,-0.002644,-0.254681
browser_version_string_mean_fake,0.200146,-0.066708,-0.066708,0.011999,0.011999,-0.052739,-0.052739,0.296407,0.083551,1.000000,0.157755,0.119712,-0.016507,-0.034325,-0.054880,0.028933,0.037274,0.007563,-0.028386


In [32]:
train['date_created'] = pd.to_datetime(train['date_created'])    
train['user_date_created'] = pd.to_datetime(train['user_date_created'])    
train['date_created_delta'] = (train['date_created'] - train['user_date_created'])  / np.timedelta64(1,'D')

In [72]:
test['date_created'] = pd.to_datetime(test['date_created'])    
test['user_date_created'] = pd.to_datetime(test['user_date_created'])    
test['date_created_delta'] = (test['date_created'] - test['user_date_created'])  / np.timedelta64(1,'D')

In [73]:
test['date_created_delta'] = (test['date_created_delta'] - test['date_created_delta'].mean()) / test['date_created_delta'].std()

In [77]:
train_columns = ['rating', 
        'user_id_mean_fake', 'user_id_count_fake',
       'company_id_mean_fake', 'company_id_count_fake',
       'user_ip_trunc_mean_fake', 'user_ip_trunc_count_fake',
'product_id_mean_fake',
       'product_id_count_fake']
#         'os_family_mean_fake',
#        'os_family_count_fake', 'os_version_string_mean_fake',
#        'os_version_string_count_fake', 'device_brand_mean_fake',
#        'device_brand_count_fake', 'device_model_mean_fake',
#        'device_model_count_fake']

In [78]:
X = train[train_columns]

In [79]:
regression_clf = LogisticRegression(C = 0.5, class_weight = {1: 150}, penalty='l1')

In [247]:
# bc_clf = BaggingClassifier(regression_clf, n_estimators=20)

In [108]:
# # comment_vect = CountVectorizer(min_df=0.05)
# comment_features = comment_vect.fit_transform(train.comment)
# agent_vect = CountVectorizer(min_df=0.2)
# agent_features = agent_vect.fit_transform(train.user_agent)

In [128]:
# X = pd.concat(
#     [
#         train[train_columns],
#         pd.DataFrame(comment_features.todense()),        
#         pd.DataFrame(agent_features.todense())
#     ],
#     axis=1
# )

In [133]:
# cross_val_score(regression_clf, 
#                 X, 
#                 train.is_fake.astype(int), 
#                 cv=5, 
#                 scoring='f1').mean()

0.8220461173062109

In [ ]:
# from sklearn.svm import SVC
# svm = SVC(C=0.5, class_weight = {1: 150})

In [80]:
regression_clf.fit(X, train.is_fake.astype(int))

LogisticRegression(C=0.5, class_weight={1: 150}, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [109]:
test_comment_features = comment_vect.transform(test.comment)
test_agent_features = agent_vect.transform(test.user_agent)

In [129]:
X_test = pd.concat(
    [
        test[train_columns],
        pd.DataFrame(test_comment_features.todense()),        
        pd.DataFrame(test_agent_features.todense())
    ],
    axis=1
)

In [81]:
predicted = regression_clf.predict_proba(test[train_columns])

In [82]:
predicted = predicted[:, 1].astype(np.float32)

In [88]:
(predicted > 0.5).sum()

1207

In [84]:
submition = pd.read_csv('/home/oleg/Downloads/dima_averaged_13_9.csv')

In [85]:
mask = (test.user_id.isin(train.user_id).astype(int) + test.user_ip_trunc.isin(train.user_ip_trunc).astype(int) + \
        test.company_id.isin(train.company_id).astype(int) + test.product_id.isin(train.product_id).astype(int) > 2)

In [86]:
len(predicted)

252099

In [87]:
submition["is_fake"][mask] = predicted

/home/oleg/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [76]:
submition.to_csv("sample_submission.csv", index=False)

In [113]:


lgb_data_train = lgb.Dataset(
    X,
    label=train.is_fake.astype(int), 
    free_raw_data=False
)

In [68]:


def lgb_f1_score(preds, dtrain):
    labels = dtrain.get_label()
    preds = preds > 0.5
    
    return "f1_score", f1_score(labels, preds), True

params = {
    "objective": "binary", 
    "metric": "binary_logloss", 
    "learning_rate": 0.02, 
    "random_state": 42,
    "scale_pos_weight": 100,
    "depth": 3,
    "num_leaves": 4
}

In [60]:
history = lgb.cv(
    params,
    lgb_data_train, 
    nfold=5, 
    num_boost_round=100000,
    early_stopping_rounds=50, 
    verbose_eval=10,
    feval=lgb_f1_score
)

[10]	cv_agg's f1_score: 0.95926 + 0.00338865	cv_agg's binary_logloss: 0.525504 + 8.24713e-06
[20]	cv_agg's f1_score: 0.95926 + 0.00338865	cv_agg's binary_logloss: 0.406974 + 1.68323e-05
[30]	cv_agg's f1_score: 0.95926 + 0.00338865	cv_agg's binary_logloss: 0.31965 + 2.49189e-05
[40]	cv_agg's f1_score: 0.95926 + 0.00338865	cv_agg's binary_logloss: 0.253569 + 3.23835e-05
[50]	cv_agg's f1_score: 0.95926 + 0.00338865	cv_agg's binary_logloss: 0.202623 + 3.94792e-05


In [120]:
gbm = lgb.train(
    params,
    lgb_data_train,
    num_boost_round=250,
    valid_sets=[lgb_data_train],
    feval=lgb_f1_score
) 

[1]	training's binary_logloss: 0.67336	training's f1_score: 0.959461
[2]	training's binary_logloss: 0.654348	training's f1_score: 0.959461
[3]	training's binary_logloss: 0.636068	training's f1_score: 0.959461
[4]	training's binary_logloss: 0.618478	training's f1_score: 0.959461
[5]	training's binary_logloss: 0.601541	training's f1_score: 0.959461
[6]	training's binary_logloss: 0.58522	training's f1_score: 0.959461
[7]	training's binary_logloss: 0.569485	training's f1_score: 0.959461
[8]	training's binary_logloss: 0.554305	training's f1_score: 0.959461
[9]	training's binary_logloss: 0.539653	training's f1_score: 0.959461
[10]	training's binary_logloss: 0.525501	training's f1_score: 0.959461
[11]	training's binary_logloss: 0.511828	training's f1_score: 0.959461
[12]	training's binary_logloss: 0.498609	training's f1_score: 0.959461
[13]	training's binary_logloss: 0.485824	training's f1_score: 0.959461
[14]	training's binary_logloss: 0.473453	training's f1_score: 0.959461
[15]	training's b

[117]	training's binary_logloss: 0.0498713	training's f1_score: 0.959461
[118]	training's binary_logloss: 0.0488803	training's f1_score: 0.959461
[119]	training's binary_logloss: 0.04791	training's f1_score: 0.959461
[120]	training's binary_logloss: 0.0469599	training's f1_score: 0.957798
[121]	training's binary_logloss: 0.0460295	training's f1_score: 0.957798
[122]	training's binary_logloss: 0.0451185	training's f1_score: 0.957798
[123]	training's binary_logloss: 0.0442264	training's f1_score: 0.957798
[124]	training's binary_logloss: 0.0433529	training's f1_score: 0.957798
[125]	training's binary_logloss: 0.0424974	training's f1_score: 0.957798
[126]	training's binary_logloss: 0.0416596	training's f1_score: 0.957798
[127]	training's binary_logloss: 0.0408392	training's f1_score: 0.957798
[128]	training's binary_logloss: 0.0400357	training's f1_score: 0.957798
[129]	training's binary_logloss: 0.0392489	training's f1_score: 0.957798
[130]	training's binary_logloss: 0.0384782	training's

[230]	training's binary_logloss: 0.00573928	training's f1_score: 0.96054
[231]	training's binary_logloss: 0.00563958	training's f1_score: 0.96054
[232]	training's binary_logloss: 0.00554188	training's f1_score: 0.96054
[233]	training's binary_logloss: 0.00544523	training's f1_score: 0.96054
[234]	training's binary_logloss: 0.00535094	training's f1_score: 0.96054
[235]	training's binary_logloss: 0.0052589	training's f1_score: 0.96054
[236]	training's binary_logloss: 0.00516773	training's f1_score: 0.96054
[237]	training's binary_logloss: 0.00507929	training's f1_score: 0.96054
[238]	training's binary_logloss: 0.00499262	training's f1_score: 0.96054
[239]	training's binary_logloss: 0.00490674	training's f1_score: 0.96054
[240]	training's binary_logloss: 0.00482346	training's f1_score: 0.96054
[241]	training's binary_logloss: 0.00474184	training's f1_score: 0.96054
[242]	training's binary_logloss: 0.00466102	training's f1_score: 0.96054
[243]	training's binary_logloss: 0.00458259	training

In [121]:
probabilities = gbm.predict(X_test)

In [123]:
(probabilities > 0.5).sum()

819

In [124]:
submition = pd.DataFrame(test.comment_id)
submition["is_fake"] = probabilities

In [125]:
submition.to_csv("sample_submission.csv", index=False)